In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


# Read in the p064_matter.csv file 
data =[]
with open("p061_matter.csv") as f:
#with open("p065_matter.csv") as f:
    data = f.readlines()
header = data[0]
data = data[1:]

In [ ]:
# Read the CCEP data into a pandas dataframe. It is in xlsx format
df = pd.read_excel(r"Z:\user home\Roberto\NeuralFigures\P061\CCEP\2023-05-22\Bipolar\Ref Gr\Peak locations and amplitudes\6mA.xlsx", sheet_name=None, header=None)
sheetNames = df.keys()
# For each sheet, extract the n1, p1, n2 and p2 data from latency and amplitude, as well as the z-score.
# Store the data in a dictionary
ccepData = []
for sheet in sheetNames:
    sheetData = df[sheet]
    addtionalLabel = ''
    newLabels = []
    for column0Info, column1Info in zip(sheetData.iloc[0], sheetData.iloc[1]):
        addtionalLabel = column0Info if pd.notna(column0Info) else addtionalLabel
        newLabel = column1Info
        if pd.notna(column1Info) and column1Info != 'Channel':
            #print(additionalLabel, column1Info)
            # Remove any whitespace from additionalLabel and column1Info
            addtionalLabel = addtionalLabel.replace(' ', '_')
            column1Info = column1Info.replace(' ', '')
            newLabel = addtionalLabel + '_' + column1Info + '_' + sheet
            print(newLabel)
            column1Info = newLabel
        newLabels.append(newLabel)
    # Replace iloc[1] with new labels
    sheetData.iloc[1] = newLabels
    sheetData.columns = sheetData.iloc[1]
    sheetData = sheetData[2:]
    # Remove duplicate columns
    sheetData = sheetData.loc[:,~sheetData.columns.duplicated()]
    # Set the PD index to the channel number
    sheetData.set_index('Channel', inplace=True)
    ccepData.append(sheetData)
# Concatenate the dataframes
ccepData = pd.concat(ccepData, axis=1)
# Remove NaN columns
ccepData = ccepData.loc[:, ccepData.columns.notnull()]
# Now get the header information from the CCEP data - just extract like Pandas would normally
dataNames = list(ccepData.columns)

In [ ]:
dataNames

In [ ]:
coords = {}
contactNames = {}
for line in data:
    line = line.split(",")
    coords[line[1]] = [float(line[4]), float(line[5]), float(line[6])]
    contactNames[line[0]] = line[1]
channelNumbers = np.arange(1, len(coords)+1) # Useful for translation between channel number and contact name

In [ ]:
# The stimulating electrodes are:
# Bipolar
# LAMY4, LAMY5
# LHH4, LHH5
# LHT4, LHT5
# LC8, LC9
# RHH1, RHH2
# RFC1, RFC2

# Monopolar
# LAMY4
# LHH4
# LHT4
# LC8
# RHH1
# RFC1

# Reference electrode is 101, which is RF3
# Other reference electrode is a scalp one, which we'll have to get manually from the eeg techs.

# We want to calculate the distance of every electrode from the stimulating electrodes
# Let's start with monopolar to keep it simple.
stimElectrodeNumbers = [1, 18, 31, 47, 85, 94] # p061
stimElectrodes = [contactNames[str(x)] for x in stimElectrodeNumbers]
#stimElectrodes = ["LAMY4", "LHH4", "LHT4", "LC8", "RHH1", "RFC1"] # p064
#stimElectrodes = ["LAMY7", "LOFCH3", "LACF2", "RAMY6", "RHH8"] p065

refElectrode = "None"
# We also want to assign unique colors to each electrode.
names = []
for line in data:
    line = line.split(",")
    name = line[1]
    # Remove the number from the electrode name
    name = ''.join([i for i in name if not i.isdigit()])
    names.append(name)

names = list(set(names)) # Reduce it down to unique names
# Now we want to assign a color to each electrode, make it a dictionary.
# Start with base colors, then combinations of colors.
colors = {}
base = 0.06
for idx, name in enumerate(names):
    colors[name] = plt.cm.gist_rainbow((base * idx))

distances =  {} # Each key will be a stimulating electrode, and the value will be a list of distances to each electrode, in order. This is a tuple with the receiving electrode name.
for stimName, stim in zip(sheetNames, stimElectrodes):
    distances[stim] = []
    for jdx, electrode in enumerate(coords.keys()):
        if electrode == refElectrode:
            continue
        
        distances[stim].append((electrode, np.linalg.norm(np.array(coords[stim]) - np.array(coords[electrode]))))
        # Add the electrode name and the distance to the ccepData dataframe
        #ccepData.insert(-1, electrode + '_Distance_' + stimName, np.linalg.norm(np.array(coords[stim]) - np.array(coords[electrode])))
    ccepData.insert(0, stim + '_' + stimName + '_Distance', [x[1] for x in distances[stim]])
ccepData.insert(0, 'ChannelName', [x[0] for x in distances[stimElectrodes[0]]])
'''
fig, ax = plt.subplots(len(distances), 1, figsize=(25, 30))
fig.suptitle("Distance from each contact to each stimulation site")
fig.tight_layout(pad=6.0)
for idx, stimElectrode in enumerate(distances):
    #distances[stimElectrode].sort(key = lambda x: x[1])
    ax[idx].bar([x[0] for x in distances[stimElectrode]], [x[1] for x in distances[stimElectrode]], width=0.4, color=[colors[''.join([i for i in x[0] if not i.isdigit()])] for x in distances[stimElectrode]])
    #plt.bar([x[0] for x in distances[stimElectrode]], [x[1] for x in distances[stimElectrode]], width=0.4)
    ax[idx].set_xticks([x[0] for x in distances[stimElectrode]])
    ax[idx].set_xticklabels([x[0] for x in distances[stimElectrode]], rotation=90)
    ax[idx].set_title("Distance from " + stimElectrode + " to each contact")
# Force all names to appear in the legend
fig.legend([plt.Line2D([0], [0], color=colors[''.join([i for i in x if not i.isdigit()])], lw=4) for x in names], names, loc="center right", title="Electrode Names")
'''

In [ ]:
fig, ax = plt.subplots(len(dataNames), 2, figsize=(50, 300))
fig.tight_layout(pad=6.0)
fullCCEPDataNames = ccepData.columns
for idx, dataName in enumerate(dataNames):
    # Sort data by Euclidean distance from the stimulating electrode
    # stimElectrodes[0] = S1, stimElectrodes[1] = S2, etc.
    stimElectrodeNum = dataName.split('_')[-1]
    # Find the index of the column that we want to sort by
    ccepData[dataName] = ccepData[dataName].fillna(0.0).infer_objects(copy=False) # Fill NaNs with 0.0, and do not infer anything. The copy=false is to avoid a warning.
    ccepData = ccepData.sort_values(by=dataName)
    leftPlot = ax[idx][0] # This will contain data sorted by the natural data order
    rightPlot = ax[idx][1] # This will contain the data sorted by euclidean distance
    leftPlot.bar(ccepData['ChannelName'], ccepData[dataName], width=0.4, color=[colors[''.join([i for i in x if not i.isdigit()])] for x in ccepData['ChannelName']])
    leftPlot.set_xticks(ccepData['ChannelName'])
    leftPlot.set_xticklabels(ccepData['ChannelName'], rotation=90)
    leftPlot.set_title(dataName)
    # Right plot (Sorted by Eulclidean distance)
    distanceColumn = ccepData.filter(like=stimElectrodeNum + '_Distance')
    stimSiteName = distanceColumn.columns[0].split('_')[0]
    ccepData.sort_values(by=distanceColumn.columns[0], inplace=True)
    rightPlot.bar(ccepData['ChannelName'], ccepData[dataName], width=0.4, color=[colors[''.join([i for i in x if not i.isdigit()])] for x in ccepData['ChannelName']])
    rightPlot.set_xticks(ccepData['ChannelName'])
    rightPlot.set_xticklabels(ccepData['ChannelName'], rotation=90)
    rightPlot.set_title(dataName + ' (Sorted by Euclidean Distance from ' + stimSiteName + ')')

# Force all names to appear in the legend
fig.legend([plt.Line2D([0], [0], color=colors[''.join([i for i in x if not i.isdigit()])], lw=4) for x in names], names, loc="center right", title="Electrode Names")

In [ ]:
# Create a figure for just the distance data
fig, ax = plt.subplots(len(sheetNames), 2, figsize=(25, 30))
fig.suptitle("Distance from each contact to each stimulation site")
fig.tight_layout(pad=6.0)
for idx, stimElectrode in enumerate(distances):
    left = ax[idx][0]
    right = ax[idx][1]
    left.bar([x[0] for x in distances[stimElectrode]], [x[1] for x in distances[stimElectrode]], width=0.4, color=[colors[''.join([i for i in x[0] if not i.isdigit()])] for x in distances[stimElectrode]])
    left.set_xticks([x[0] for x in distances[stimElectrode]])
    left.set_xticklabels([x[0] for x in distances[stimElectrode]], rotation=90)
    left.set_title("Distance from " + stimElectrode + " to each contact")
    # Right plot (Sorted by Eulclidean distance)
    distances[stimElectrode].sort(key = lambda x: x[1])
    right.bar([x[0] for x in distances[stimElectrode]], [x[1] for x in distances[stimElectrode]], width=0.4, color=[colors[''.join([i for i in x[0] if not i.isdigit()])] for x in distances[stimElectrode]])
    right.set_xticks([x[0] for x in distances[stimElectrode]])
    right.set_xticklabels([x[0] for x in distances[stimElectrode]], rotation=90)
    right.set_title("Distance from " + stimElectrode + " to each contact (Sorted by Euclidean Distance)")
fig.legend([plt.Line2D([0], [0], color=colors[''.join([i for i in x if not i.isdigit()])], lw=4) for x in names], names, loc="center right", title="Electrode Names")